## Overview

(drafting)

Prerequisites for deployment to real device

* Provisioned device
* Registered Data source

## Preparation

In [ ]:
# Import libraries

import sys
import os
import json
import glob
import tarfile

import boto3
import sagemaker
import IPython
import gluoncv

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

In [ ]:
# Initialize variables and configurations

boto3_session = boto3.session.Session()
sm_session = sagemaker.Session()

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3_session.region_name
s3_bucket = sm_session.default_bucket()
sm_role = sagemaker.get_execution_role()

print( "account_id :", account_id )
print( "region :", region )
print( "s3_bucket :", s3_bucket )
print( "sm_role :", sm_role )

In [ ]:
app_name = "lab1"
code_package_name = f"{app_name}_code"
code_package_version = "1.0"

## Create a simplest Panorama application - "Hello World!"

In [ ]:
!panorama-cli init-project --name {app_name}

In [ ]:
!cd {app_name} && panorama-cli create-package --name {code_package_name} --type Container --version {code_package_version}

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> create/edit application source code, code package descriptor</div>

1. Create a Python source code "lab1/packages/{account_id}-lab1_code-1.0/src/app.py" with following single line of code. (See: "app-v1.py"):

    ```
    print("Hello World!")
    ```


2. Edit "lab1/packages/{account_id}-lab1_code-1.0/descriptor.json", and replace <entry_file_name_under_src> with "app.py"

    ```
    "name": "/panorama/app.py"
    ```


In [ ]:
# Confirming the files are correctly editted.

source_filename = f"./{app_name}/packages/{account_id}-{code_package_name}-{code_package_version}/src/app.py"
descriptor_filename = f"./{app_name}/packages/{account_id}-{code_package_name}-{code_package_version}/descriptor.json"

print("Python source file:", source_filename)
print("Code descriptor file:", descriptor_filename)

assert os.path.exists(source_filename), f"Please create {source_filename}."

with open(descriptor_filename) as fd:
    d = json.loads( fd.read() )
    assert d["runtimeDescriptor"]["entry"]["name"] == "/panorama/app.py", f"Please edit {descriptor_filename} properly."

#### Build container image

Building docker image and adding it to the code package as the asset. This process takes some time (~ 5 min)

Note : Test Utility doesn't use the container image itself, but uses graph information.

In [ ]:
!cd {app_name} && panorama-cli build-container --container-asset-name code --package-path packages/{account_id}-{code_package_name}-{code_package_version}

## Run the Hello World application with "Test Utility"

In [ ]:
%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename}


## Simple video pass-through app

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> edit the application source code</div>

1. Edit "lab1/packages/357984623133-lab1_code-1.0/src/app.py". You can copy & paste from "app-v2.py".


#### Add camera input node and HDMI output node

In [ ]:
camera_node_name = f"{app_name}_camera"
data_sink_node_name = f"{app_name}_data_sink"

In [ ]:
!cd {app_name} && panorama-cli add-panorama-package --type camera --name {camera_node_name}

In [ ]:
!cd {app_name} && panorama-cli add-panorama-package --type data_sink --name {data_sink_node_name}

#### Manually edit the graph.json and connect nodes

1. Open "lab1/graphs/lab1/graph.json" by text editor.
2. Add edges to connect camera -> code, and code -> hdmi.

    ```
        "edges": [
            {
                "producer": "lab1_camera.video_out",
                "consumer": "code_node.video_in"
            },
            {
                "producer": "code_node.video_out",
                "consumer": "lab1_data_sink.video_in"
            }            
        ]
    ```


## Run the Video pass-through application with "Test Utility"

In [ ]:
video_filepath = "../../videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--camera-node-name lab1_camera \
--video-file {video_filepath} \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S

In [ ]:
# View latest screenshot image

latest_screenshot_dirname = sorted( glob.glob( "./screenshots/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

## Extend the pass-through application to real Computer Vision application

#### Export "yolo3_mobilenet1.0_coco" from Gluoncv model zoo

In [ ]:
# Initialize model related variables and configurations

model_package_name = f"{app_name}_model"
model_package_version = "1.0"

people_detection_model_name = "people_detection_model"

In [ ]:
def export_model_and_create_targz( prefix, name, model ):
    os.makedirs( prefix, exist_ok=True )
    gluoncv.utils.export_block( os.path.join( prefix, name ), model, preprocess=False, layout="CHW" )

    tar_gz_filename = f"{prefix}/{name}.tar.gz"
    with tarfile.open( tar_gz_filename, "w:gz" ) as tgz:
        tgz.add( f"{prefix}/{name}-symbol.json", f"{name}-symbol.json" )
        tgz.add( f"{prefix}/{name}-0000.params", f"{name}-0000.params" )
        
    print( f"Exported : {tar_gz_filename}" )

In [ ]:
# Export object detection model. Reset the classes for human detection only.
people_detection_model = gluoncv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
people_detection_model.reset_class(["person"], reuse_weights=['person'])
export_model_and_create_targz( "models", "yolo3_mobilenet1.0_coco_person", people_detection_model )


#### Add exported model files in the model package

In [ ]:
!cd {app_name} && panorama-cli create-package --name {model_package_name} --type Model --version {code_package_version}

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> edit model descriptor file</div>

1. Edit "lab1/packages/{account_id}-lab1_model-1.0/descriptor.py" with following values:

```
"framework": "MXNET",
"inputs": [
    {
        "name": "data",
        "shape": [
            1, 3, 480, 600
        ]
    }
]
```


In [ ]:
# Confirming the file is correctly editted.

descriptor_filename = f"./{app_name}/packages/{account_id}-{model_package_name}-{model_package_version}/descriptor.json"

print("Model descriptor file:", descriptor_filename)

with open(descriptor_filename) as fd:
    d = json.loads( fd.read() )
    assert d["mlModelDescriptor"]["framework"] == "MXNET", f"Please edit {descriptor_filename} properly."
    assert d["mlModelDescriptor"]["inputs"][0]["name"] == "data", f"Please edit {descriptor_filename} properly."
    assert d["mlModelDescriptor"]["inputs"][0]["shape"] == [1,3,480,600], f"Please edit {descriptor_filename} properly."


#### Add the model in the model package

In [ ]:
!cd {app_name} && panorama-cli add-raw-model \
    --model-asset-name {people_detection_model_name} \
    --model-local-path ../models/yolo3_mobilenet1.0_coco_person.tar.gz \
    --descriptor-path packages/{account_id}-{model_package_name}-{model_package_version}/descriptor.json \
    --packages-path packages/{account_id}-{model_package_name}-{model_package_version}

#### Compile the model to locally run with Test Utility

In [ ]:
people_detection_model_data_shape = '{"data":[1,3,480,600]}'

%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{s3_bucket}/panorama-workshop/{app_name} \
\
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
--model-data-shape '{people_detection_model_data_shape}' \
--model-framework MXNET

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> edit the application source code</div>

1. Edit the source code "lab1/packages/{account_id}-lab1_code-1.0/src/app.py" referring to "app-v3.py":


## Run the People detection application with "Test Utility"

In [ ]:
video_filepath = "../../videos/TownCentreXVID.avi"

%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--py-file {source_filename} \
\
--model-package-name {model_package_name} \
--model-node-name {people_detection_model_name} \
--model-file-basename ./models/yolo3_mobilenet1.0_coco_person \
\
--camera-node-name lab1_camera \
\
--video-file {video_filepath} \
--video-start 0 \
--video-stop 10 \
--video-step 1 \
\
--output-screenshots ./screenshots/%Y%m%d_%H%M%S


In [ ]:
# View latest screenshot image

latest_screenshot_dirname = sorted( glob.glob( "./screenshots/*" ) )[-1]
screenshot_filename = sorted( glob.glob( f"{latest_screenshot_dirname}/*.png" ) )[-1]

print(screenshot_filename)
IPython.display.Image( filename = screenshot_filename )

## Deploy the People detection application to your Panorama appliance device

In [ ]:
# Confirming you have provisioned the device with this AWS account

panorama_client = boto3.client( "panorama" )
response = panorama_client.list_devices()

assert len(response["Devices"]) > 0, "No device found. Please provision your device with this AWS account x region."
assert len(response["Devices"]) == 1, "More than one device found. Please choose one and delete other devices."
assert response["Devices"][0]["ProvisioningStatus"] == "SUCCEEDED", "The device is not provisioned successfully."

device_id = response["Devices"][0]["DeviceId"]

print( "Device ID :", device_id )

<div class="alert alert-block alert-warning"><b>Manual edit needed:</b> edit the Docker file, to package additional libraries.</div>

1. Open ./lab1/packages/{account_id}-lab1_code-1.0/Dockerfile with text editor.
2. Add following line at the end of the file, and save.
    
    ```
    RUN pip3 install opencv-python boto3
    ```


#### Build the container image again with latest code

In [ ]:
!cd {app_name} && panorama-cli build-container --container-asset-name code --package-path packages/{account_id}-{code_package_name}-{code_package_version}

#### Package application (upload locally prepared packages onto Cloud)

In [ ]:
!cd {app_name} && panorama-cli package-application

## Deploy the application using the AWS Management Console

1. Open ./lab1/graphs/lab1/graph.json with Text editor. You will need to copy & paste the contents later.
1. Open https://console.aws.amazon.com/panorama/home#deployed-applications
1. Click "Deploy aplication"
1. Follow the instruction. Choose the data source (camera) you registered, and IAM Role "XYZ" (FIXME: add Device role in the CF template)
1. Wait until the deployment finishes
1. Confirm the application status on the Deployed applications becomes "Running" status.

## Check HDMI output (if you have HDMI display with the Panorama appliance)

1. Connect your HDMI display and cable with the Panorama appliance device.
2. Confirm that camera image and bounding boxes are visible on the display.

## Check application logs on CloudWatch Logs

1. Open https://console.aws.amazon.com/panorama/home#deployed-applications
1. Click the deployed application
1. Locate the application instance ID ("UUID")
1. Open https://console.aws.amazon.com/cloudwatch/home#logsV2:log-groups
1. Search for a log group which contains the application instance ID, click it.
1. Find a log stream "console_output", click it.
1. Confirm logs from application are visible. 

## Delete the application

1. Open https://console.aws.amazon.com/panorama/home#deployed-applications
1. Select the application.
1. From the "Actions" drop-down menu, choose "Delete from device"
1. Follow the instruction
1. Wait until the application disappears from the list.